# Speech to Speech Flow

## Speech to Text
Objectives:
1. Transcribe text　✅
2. Preserve timestamp　✅

In [1]:
from pathlib import Path
import pytube as pt
from openai import OpenAI
import os
from dotenv import load_dotenv

dotenv_path = Path("__file__").resolve().parents[1].parents[0] / '.local.env'
load_dotenv(dotenv_path)

OpenAI.api_key = os.getenv("OPENAI_API_KEY")
data_path = Path("__file__").resolve().parents[1].parents[0] / "local_data"

In [2]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=g_ltie2ZGNY"

In [3]:
yt = pt.YouTube(YOUTUBE_VIDEO_URL)
stream = yt.streams.filter(only_audio=True)[0]
audio_file_path = os.path.join(data_path, "rakugo_v1.mp3")
stream.download(filename=audio_file_path)

'/Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1.mp3'

In [4]:
# download video as well
video_file_path = os.path.join(data_path, "rakugo_v1.mp4")
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').asc().first().download(filename=video_file_path)

'/Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1.mp4'

In [5]:
# take first 1 minutes of the audio
from pydub import AudioSegment
audio = AudioSegment.from_file(audio_file_path)
duration = 60 * 1000

audio = audio[:duration]
shortened_audio_file_path = os.path.join(data_path, "rakugo_v1_shortened.mp3")
audio.export(shortened_audio_file_path, format="mp3")

<_io.BufferedRandom name='/Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1_shortened.mp3'>

In [6]:
# trim video as well
from moviepy.editor import VideoFileClip
video = VideoFileClip(video_file_path)
video = video.subclip(0, int(duration/1000))
shortened_video_file_path = os.path.join(data_path, "rakugo_v1_shortened.mp4")
video.write_videofile(shortened_video_file_path)

Moviepy - Building video /Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1_shortened.mp4.
MoviePy - Writing audio in rakugo_v1_shortenedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1_shortened.mp4



Moviepy - Done !
Moviepy - video ready /Users/howardtangkulung/code/personal_projects/rakugo/local_data/rakugo_v1_shortened.mp4


In [7]:
client = OpenAI()

shortened_audio_file_path = os.path.join(data_path, "rakugo_v1_shortened.mp3")
audio_file= open(shortened_audio_file_path, "rb")

transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)

In [8]:
print(len(transcript.segments), "segments")
print(transcript.segments[:2])

7 segments
[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 15.5600004196167, 'text': 'me', 'tokens': [50364, 1398, 51142], 'temperature': 0.20000000298023224, 'avg_logprob': -0.6330402493476868, 'compression_ratio': 1.0288461446762085, 'no_speech_prob': 0.2711162269115448}, {'id': 1, 'seek': 0, 'start': 16.040000915527344, 'end': 23.0, 'text': 'a 練習したような小刻みの白松がございます 旬風ていちの助と申しましてまぁ', 'tokens': [51166, 64, 220, 47027, 34025, 8533, 17010, 3203, 7322, 45500, 11362, 2972, 13558, 36897, 5142, 43808, 220, 4479, 105, 22713, 2996, 1764, 6574, 2972, 37618, 3193, 3526, 111, 45349, 8822, 37566, 51514], 'temperature': 0.20000000298023224, 'avg_logprob': -0.6330402493476868, 'compression_ratio': 1.0288461446762085, 'no_speech_prob': 0.2711162269115448}]


In [9]:
# remove alphanumeric characters with regex
import re

segments = []
for segment in transcript.segments:
    text = re.sub(r'[a-zA-Z0-9🎶]', '', segment["text"]).strip()
    start = segment["start"]
    end = segment["end"]

    if text and (end-start) > 2.5:
        segments.append({
            "text": text,
            "start": start,
            "end": end
        })

for i in range(len(segments[:5])):
    print(segments[i])

{'text': '練習したような小刻みの白松がございます 旬風ていちの助と申しましてまぁ', 'start': 16.040000915527344, 'end': 23.0}
{'text': '嘘つきは泥棒の始まりなんと まあねほんと猫立ちの悪い嘘つくよりはちょいと間抜けな泥棒のほうが', 'start': 23.0, 'end': 30.920000076293945}
{'text': '会があるようでございますおしまいしまい こっち来いしまいはい親分なんかご用ですかよですかじゃないよね', 'start': 30.920000076293945, 'end': 38.15999984741211}
{'text': 'おめのこと仲間がなぁなんて言ってるか知ってるのかねあら見込みがねからさっさと足 荒らして敵に戻したらどうだってみんな濡れるとのこと言ってんだよねどうするよ', 'start': 38.15999984741211, 'end': 46.15999984741211}
{'text': '泥棒やめるか足洗うかなんて言ってんすか 自分でもまあ土地ばかり踏んでんなってのはよくわかってるんですかね', 'start': 46.63999938964844, 'end': 53.400001525878906}


## Text to text
Objectives
1. Easify the text ✅
2. Try to preserve the length of text ✅

In [10]:
sentences = [segment["text"] for segment in segments]

In [11]:
# join segements in the format of
# 1. segment1
# 2. segment2
# ...
input_text = "\n".join([f"{i+1}. {sentence}" for i, sentence in enumerate(sentences)])
print(input_text)
JLPT_LEVEL = "N4"

1. 練習したような小刻みの白松がございます 旬風ていちの助と申しましてまぁ
2. 嘘つきは泥棒の始まりなんと まあねほんと猫立ちの悪い嘘つくよりはちょいと間抜けな泥棒のほうが
3. 会があるようでございますおしまいしまい こっち来いしまいはい親分なんかご用ですかよですかじゃないよね
4. おめのこと仲間がなぁなんて言ってるか知ってるのかねあら見込みがねからさっさと足 荒らして敵に戻したらどうだってみんな濡れるとのこと言ってんだよねどうするよ
5. 泥棒やめるか足洗うかなんて言ってんすか 自分でもまあ土地ばかり踏んでんなってのはよくわかってるんですかね
6. これから真心に立ち帰った悪事に励みますから 今までより置いちゃってくださいこの通りで言ってること


In [12]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": """
    日本の落語の一節と日本語能力試験（JLPT）のレベルが与えられます。\n
    その節を、指定されたJLPTレベルに適した語彙を使って簡単にしてください。\n
    語彙のみを変更してください。文の構造や意味は変更しないでください。\n
     """},
    {"role": "user", "content": f"""
    JLPTレベル: {JLPT_LEVEL}\n
     落語の節:\n
     {input_text}
     """}
  ]
)


In [13]:
completion.choices[0].message

ChatCompletionMessage(content='\nJLPTレベル: N4\n\n落語の節:\n\n1. 練習したような小さい白松がいます 名前は旬風ていちの助と言います\n2. 嘘つきは泥棒の始まりとは そうですね本当に、立ちまわりが悪い嘘つくよりは少し間抜けな泥棒の方が\n3. 集まりがあるようですね終わり終わり こっちきて 終わり終わり、はいリーダー何か仕事ですか？仕事ですか？ではなくてね\n4. あなたの事、友達が何を言ってるか知ってますか？あら、見込みがないから早く足を速くして敵に戻せとみんなが愚痴ってるんだよね、どうする？\n5. 泥棒をやめるか、足洗うか、そう言ってますか？ 自分でもまあ地面ばかり踏み固めてるってのはよくわかってるの？\n6. これから真心を持って悪行に専念しますから 今までより忘れてください。これで同意していることを語ってるんです。', role='assistant', function_call=None, tool_calls=None)

In [14]:
completion_output = completion.choices[0].message.content
print(completion_output)


JLPTレベル: N4

落語の節:

1. 練習したような小さい白松がいます 名前は旬風ていちの助と言います
2. 嘘つきは泥棒の始まりとは そうですね本当に、立ちまわりが悪い嘘つくよりは少し間抜けな泥棒の方が
3. 集まりがあるようですね終わり終わり こっちきて 終わり終わり、はいリーダー何か仕事ですか？仕事ですか？ではなくてね
4. あなたの事、友達が何を言ってるか知ってますか？あら、見込みがないから早く足を速くして敵に戻せとみんなが愚痴ってるんだよね、どうする？
5. 泥棒をやめるか、足洗うか、そう言ってますか？ 自分でもまあ地面ばかり踏み固めてるってのはよくわかってるの？
6. これから真心を持って悪行に専念しますから 今までより忘れてください。これで同意していることを語ってるんです。


In [15]:
# save the easified text using regex
# "number". "sentence" -> "sentence"
import re

easified_sentences = re.findall(r"\d+\. (.+)", completion_output)
print(len(easified_sentences), " sentences found from ", len(segments), " sentences")
print(easified_sentences)

6  sentences found from  6  sentences
['練習したような小さい白松がいます 名前は旬風ていちの助と言います', '嘘つきは泥棒の始まりとは そうですね本当に、立ちまわりが悪い嘘つくよりは少し間抜けな泥棒の方が', '集まりがあるようですね終わり終わり こっちきて 終わり終わり、はいリーダー何か仕事ですか？仕事ですか？ではなくてね', 'あなたの事、友達が何を言ってるか知ってますか？あら、見込みがないから早く足を速くして敵に戻せとみんなが愚痴ってるんだよね、どうする？', '泥棒をやめるか、足洗うか、そう言ってますか？ 自分でもまあ地面ばかり踏み固めてるってのはよくわかってるの？', 'これから真心を持って悪行に専念しますから 今までより忘れてください。これで同意していることを語ってるんです。']


## Text to speech
Objectives
1. Synchronise with timestamps ✅
2. Use same voice ✅

In [16]:
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

In [17]:
audio_file_paths = [str(shortened_audio_file_path)]

In [18]:
from elevenlabs import clone, generate, play

voice = clone(
    api_key=ELEVENLABS_API_KEY,
    name="rakugo_v1",
    files=audio_file_paths,
)

In [19]:
audios = []
for sentence in easified_sentences:
    audio = generate(
        api_key=ELEVENLABS_API_KEY,
        text=sentence,
        voice=voice,
        model="eleven_multilingual_v2",
        output_format="mp3_44100_128"
    )
    audios.append(audio)

In [20]:
# play(audios[0])

In [21]:
temp_res_file_path = os.path.join(data_path, "temp_results")
os.makedirs(temp_res_file_path, exist_ok=True)

In [22]:
# save generated audio to mp3
for i, audio in enumerate(audios):
    audio_file_path = os.path.join(temp_res_file_path, f"rakugo_v1_{i}.mp3")
    with open(audio_file_path, "wb") as f:
        f.write(audio)
    print(f"audio file saved to {audio_file_path}")

audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_0.mp3
audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_1.mp3
audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_2.mp3
audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_3.mp3
audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_4.mp3
audio file saved to /Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_5.mp3


In [23]:
# speed up audio according to segments start and end

from pydub import AudioSegment
from pydub.effects import speedup

for i, segment in enumerate(segments):
    audio_file_path = os.path.join(temp_res_file_path, f"rakugo_v1_{i}.mp3")
    audio = AudioSegment.from_file(audio_file_path, format="mp3")
    audio_dur = len(audio) / 1000
    speed = audio_dur / (segment['end'] - segment['start'])
    print("original audio duration", audio_dur, "segment duration", segment['end'] - segment['start'], "speed", speed)
    # round speed to nearest from 1, 1.25, 1.5
    speed = round(speed*4) / 4
    if speed <= 1:
        speed = 1
        speeded_audio = audio
    else:
        speeded_audio = speedup(audio, speed)
    print(f"speeded audio with speed {speed}")
    speeded_audio.export(os.path.join(temp_res_file_path, f"rakugo_v1_{i}_speeded.mp3"), format="mp3")
    print(f"audio file saved")

original audio duration 4.911 segment duration 6.959999084472656 speed 0.7056035410918585
speeded audio with speed 1
audio file saved
original audio duration 8.96 segment duration 7.920000076293945 speed 1.1313131204151086
speeded audio with speed 1.25
audio file saved
original audio duration 7.784 segment duration 7.239999771118164 speed 1.0751381555358557
speeded audio with speed 1
audio file saved
original audio duration 9.744 segment duration 8.0 speed 1.218
speeded audio with speed 1.25
audio file saved
original audio duration 6.165 segment duration 6.760002136230469 speed 0.9119819603249038
speeded audio with speed 1
audio file saved
original audio duration 7.602 segment duration 8.680000305175781 speed 0.8758064208208631
speeded audio with speed 1
audio file saved


In [26]:
# merge speeded audio according to segments start and end
# use the first segment start time as the start time of the merged audio
# fill first silence with original audio
for i, segment in enumerate(segments):
    audio_file_path = os.path.join(temp_res_file_path, f"rakugo_v1_{i}_speeded.mp3")
    original_audio_file_path = os.path.join(data_path, "rakugo_v1_shortened.mp3")
    original_audio = AudioSegment.from_file(original_audio_file_path, format="mp3")
    audio = AudioSegment.from_file(audio_file_path, format="mp3")
    current_time = segment['start'] * 1000
    print("current_time", current_time, "segment start", segment['start'], "segment end", segment['end'])
    if i == 0:
        fill_dur = current_time
        fill = original_audio[:fill_dur]
        merged_audio = fill + audio
    else:
        silence_dur = segment['start'] * 1000 - current_time
        silence = AudioSegment.silent(duration=silence_dur)
        merged_audio = merged_audio + silence + audio
    current_time = len(merged_audio)

merged_audio.export(os.path.join(temp_res_file_path, f"rakugo_v1_merged.mp3"), format="mp3")

current_time 16040.000915527344 segment start 16.040000915527344 segment end 23.0
current_time 23000.0 segment start 23.0 segment end 30.920000076293945
current_time 30920.000076293945 segment start 30.920000076293945 segment end 38.15999984741211
current_time 38159.99984741211 segment start 38.15999984741211 segment end 46.15999984741211
current_time 46639.99938964844 segment start 46.63999938964844 segment end 53.400001525878906
current_time 53400.001525878906 segment start 53.400001525878906 segment end 62.08000183105469


<_io.BufferedRandom name='/Users/howardtangkulung/code/personal_projects/rakugo/local_data/temp_results/rakugo_v1_merged.mp3'>

In [27]:
import moviepy.editor as mp

final_res_file_path = os.path.join(data_path, "final_results")

audio = mp.AudioFileClip(os.path.join(temp_res_file_path, "rakugo_v1_merged.mp3"))
video1 = mp.VideoFileClip(shortened_video_file_path)
final = video1.set_audio(audio)

final.write_videofile(os.path.join(final_res_file_path, "rakugo_v1_final.mp4"))

Moviepy - Building video /Users/howardtangkulung/code/personal_projects/rakugo/local_data/final_results/rakugo_v1_final.mp4.
MoviePy - Writing audio in rakugo_v1_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /Users/howardtangkulung/code/personal_projects/rakugo/local_data/final_results/rakugo_v1_final.mp4



Moviepy - Done !
Moviepy - video ready /Users/howardtangkulung/code/personal_projects/rakugo/local_data/final_results/rakugo_v1_final.mp4


In [28]:
import glob

files = glob.glob(os.path.join(temp_res_file_path, "*.mp3"))
for f in files:
    os.remove(f)